# Instance 11

In [1]:
import os
from pyomo.environ import *
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Importing DataSet

In [2]:
file_path = 'Instance_11.xlsx'
# Read the Excel file into a dictionary of DataFrames, where keys are sheet names
Instance_data = pd.read_excel(file_path, sheet_name=None)

In [3]:
# Exam Data Set

Exams = Instance_data['Sheet1']
Exams['Exams'] = Exams['Exams'] - 1
ExamsList = list(Exams['Exams'])

### Dictionary of Students and their related Exams

In [4]:
# Creating a Dictionary of Students and Their Related Exams
Students_exams = Instance_data['Sheet3']
Students_exams['ExamNumber'] = Students_exams['ExamNumber'] - 1
# Initialize an empty dictionary to store the students and their exams
students = {}
# Iterate over the rows in the DataFrame
for index, row in Students_exams.iterrows():
    student = row['Student']
    exam_number = row['ExamNumber']
    
    # Check if the student is already in the dictionary
    if student in students:
        students[student].append(exam_number)
    # If the student is not in the dictionary, create a new key-value pair    
    else:
        students[student] = [exam_number]

### TimeSlots

In [5]:
Time_s = Instance_data['Sheet2']
time_slots = Time_s.loc[0, 'TimeSlot']
TimeList= list(range (time_slots))

In [6]:
TimeList

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34]

### Distance

In [7]:
#Distance
Distance=list(range(1,6))
Distance

[1, 2, 3, 4, 5]

###  Finding Conflicted Exams (Weighted-Unweighted)

In [8]:
# Unweighted
# Step 1: Initialize the dictionary to store conflicting exams
conflicting_exams = {exam: set() for exams_list in students.values() for exam in exams_list}

# Step 2: Populate the dictionary with conflicting exams
for student_exams in students.values():
    for i, exam1 in enumerate(student_exams):
        for exam2 in student_exams[i + 1:]:
            conflicting_exams[exam1].add(exam2)
            conflicting_exams[exam2].add(exam1)

# Step 3: Create the unweighted conflicting matrix
Unweighted_Conflict_Matrix = [
    [1 if exam2 in conflicting_exams[exam1] else 0 for exam2 in ExamsList] for exam1 in ExamsList
]


In [9]:
# Weighted

# The weighted Conflicting MATRIX
# Step 1: Initialize the dictionary to store conflicting exams
conflicting_exams = {exam: set() for exams_list in students.values() for exam in exams_list}

# Step 2: Populate the dictionary with conflicting exams
for student_exams in students.values():
    for i, exam1 in enumerate(student_exams):
        for exam2 in student_exams[i + 1:]:
            conflicting_exams[exam1].add(exam2)
            conflicting_exams[exam2].add(exam1)

# Step 3: Create the weighted conflicting matrix
weighted_conflicting_matrix = [[0 for _ in ExamsList] for _ in ExamsList]

# Step 4: Calculate the number of students in each conflicting exam pair and fill the matrix
for i, exam1 in enumerate(ExamsList):
    for j, exam2 in enumerate(ExamsList):
        if exam2 in conflicting_exams[exam1]:
            count = sum(1 for student_exams in students.values() if exam1 in student_exams and exam2 in student_exams)
            weighted_conflicting_matrix[i][j] = count



### Converting the Llist of UC and WC to Matrix of Coefficients

In [10]:
#Conflicting Exams (Weighted and Unweighted)

UC_Matrix=np.array(Unweighted_Conflict_Matrix )
WC_Matrix=np.array(weighted_conflicting_matrix)

In [11]:
UC_Matrix

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0]])

In [12]:
WC_Matrix

array([[ 0,  0, 12, ...,  0,  0,  0],
       [ 0,  0,  8, ...,  0,  0,  0],
       [12,  8,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  4,  2],
       [ 0,  0,  0, ...,  4,  0,  4],
       [ 0,  0,  0, ...,  2,  4,  0]])

In [13]:
# Setting up the environment


env = gp.Env(
    r"C:\Users\ghadi\Downloads\Discrete Optimization Project - Gurobi- Instance 11\Discrete Optimization Project - Instance 11-Log File.log")
env.start()

# Environment parameters
env.setParam("Threads", 1)  # Controls the number of threads to apply to parallel algorithms
env.setParam("Presolve", 1)  # Controls the presolve level. conservative (1).
env.setParam("MIPGap", 1e-4)
env.setParam('Method', 0)  # Algorithm used to solve the initial root relaxation of a MIP model. 0=primal simplex.
env.setParam("TimeLimit", 1200)  # 20 minutes time limit
env.setParam("PreSparsify", 1) # to reduce the memory used


Set parameter Username
Set parameter LogFile to value "C:\Users\ghadi\Downloads\Discrete Optimization Project - Gurobi- Instance 11\Discrete Optimization Project - Instance 11-Log File.log"
Academic license - for non-commercial use only - expires 2024-07-07
Set parameter Threads to value 1
Set parameter Presolve to value 1
Set parameter Method to value 0
Set parameter TimeLimit to value 1200
Set parameter PreSparsify to value 1


### Defining Gurobi Model

In [14]:
model = gp.Model("Exam_Scheduling", env=env)

### Defining Decision Variable and Auxiliary Variable

In [15]:
#Decision Variable
X = model.addVars(ExamsList, TimeList , vtype=GRB.BINARY, name="X")

In [16]:
# Binary auxiliary variable for controlling conflict
Y = model.addVars(ExamsList, ExamsList, Distance, vtype=GRB.BINARY, name="Y")

###  Defining Constraints 

In [17]:
#Constraint 1
model.addConstrs(UC_Matrix[i, j] * (X[i, k] + X[j, k]) <= 1 for i in ExamsList for j in ExamsList for k in TimeList)
model.update()

In [18]:
#Constraint 2
model.addConstrs(sum(X[i, k] for k in TimeList) == 1 for i in ExamsList for k in TimeList)
model.update()

In [19]:
#Constraint C3
model.addConstrs((X[i, k] + X[j, k + d]) <= Y[i, j, d] + 1 for i in ExamsList for j in ExamsList for k in TimeList for d in Distance if (k + d) in TimeList and UC_Matrix[i, j] != 0)
model.update()

### Defining Objective Function

In [20]:
# Create the objective expression using a loop
objective_expr = 0
for i in ExamsList:
    for j in ExamsList:
        for d in Distance:
            objective_expr += Y[i, j, d] * WC_Matrix[i, j] * ((2 ** (5 - d))/16925)

# Set up the objective function for minimization
model.setObjective(objective_expr, GRB.MINIMIZE)

In [21]:
# Time limit for the model
model.Params.TimeLimit = 1200

In [22]:
model.update()

In [23]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 25843690 rows, 2349490 columns and 33630850 nonzeros
Model fingerprint: 0x687e3a9f
Variable types: 0 continuous, 2349490 integer (2349490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-05, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 14192360 rows and 2027480 columns (presolve time = 5s) ...
Presolve removed 14192360 rows and 2027616 columns (presolve time = 11s) ...
Presolve removed 15258902 rows and 2027616 columns (presolve time = 15s) ...
Presolve removed 15258902 rows and 2027620 columns (presolve time = 20s) ...
Presolve removed 15258902 rows and 2027620 columns (presolve time = 25s) ...
Presolve removed 15258902 rows and 2027620 columns (presolve time = 30s

  181018    1.7379369e-01   0.000000e+00   5.926127e+05    685s
  184028    1.7500420e-01   0.000000e+00   7.002251e+05    691s
  187038    1.7601186e-01   0.000000e+00   4.319788e+05    696s
  190048    1.7746507e-01   0.000000e+00   4.420588e+05    702s
  193058    1.7883616e-01   0.000000e+00   9.832310e+05    711s
  196068    1.8031890e-01   0.000000e+00   3.418420e+06    717s
  199078    1.8152882e-01   0.000000e+00   7.947236e+05    723s
  202088    1.8286917e-01   0.000000e+00   7.724422e+06    730s
  205098    1.8393360e-01   0.000000e+00   2.734306e+06    737s
  208108    1.8505295e-01   0.000000e+00   7.359548e+05    743s
  211118    1.8615521e-01   0.000000e+00   3.788523e+05    749s
  214128    1.8710013e-01   0.000000e+00   5.120127e+05    756s
  217138    1.8783841e-01   0.000000e+00   2.560399e+05    762s
  220148    1.8903569e-01   0.000000e+00   3.483542e+05    768s
  223158    1.8976911e-01   0.000000e+00   3.111801e+05    774s
  226168    1.9060146e-01   0.000000e+00

In [24]:
if model.Status == gp.GRB.TIME_LIMIT:

    # Extend the time limit
    extended_time_limit =2400 # Set the extended time limit in seconds
    model.setParam('TimeLimit', extended_time_limit)

    # Continue the optimization
    model.optimize()

Set parameter TimeLimit to value 2400
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 25843690 rows, 2349490 columns and 33630850 nonzeros
Model fingerprint: 0x687e3a9f
Variable types: 0 continuous, 2349490 integer (2349490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-05, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 9722122 rows, 321750 columns, 29616296 nonzeros

Continuing optimization...

     0     0          -    0               -    0.00000      -     - 3600s

Explored 1 nodes (1524560 simplex iterations) in 2400.30 seconds (4560.08 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Time limit reached
Best objective -, best bound 0.000000000000e+00, gap -


In [25]:
if model.Status == gp.GRB.TIME_LIMIT:

    # Extend the time limit
    extended_time_limit =2400 # Set the extended time limit in seconds
    model.setParam('TimeLimit', extended_time_limit)

    # Continue the optimization
    model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 25843690 rows, 2349490 columns and 33630850 nonzeros
Model fingerprint: 0x687e3a9f
Variable types: 0 continuous, 2349490 integer (2349490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-05, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 9722122 rows, 321750 columns, 29616296 nonzeros

Continuing optimization...

     0     0          -    0               -    0.00000      -     - 6000s

Explored 1 nodes (2622722 simplex iterations) in 2400.36 seconds (4619.24 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Time limit reached
Best objective -, best bound 0.000000000000e+00, gap -


In [26]:
if model.Status == gp.GRB.TIME_LIMIT:

    # Extend the time limit
    extended_time_limit =3600 # Set the extended time limit in seconds
    model.setParam('TimeLimit', extended_time_limit)

    # Continue the optimization
    model.optimize()

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 25843690 rows, 2349490 columns and 33630850 nonzeros
Model fingerprint: 0x687e3a9f
Variable types: 0 continuous, 2349490 integer (2349490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-05, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 9722122 rows, 321750 columns, 29616296 nonzeros

Continuing optimization...

     0     0          -    0               -    0.00000      -     - 9601s

Explored 1 nodes (4623887 simplex iterations) in 3600.47 seconds (9702.19 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Time limit reached
Best objective -, best bound 0.000000000000e+00, gap -


In [27]:
if model.Status == GRB.OPTIMAL:
    print("Optimization was successful and an optimal solution was found.")
elif model.Status == GRB.INFEASIBLE:
    print("The model is infeasible - no feasible solution exists.")
elif model.Status == GRB.UNBOUNDED:
    print("The model is unbounded - the objective can be improved without limit.")
print("Objective Value:", model.objVal)

Objective Value: inf


In [28]:
print("Objective Value:", model.objVal)

Objective Value: inf
